In [1]:
import ee,geemap
import numpy as np
Map = geemap.Map()
Map.add_basemap()
sh_cn = ee.Geometry.Rectangle([70, 0, 138, 55], None, False)
sh = ee.Geometry.Rectangle([-180, -90, 180, 90], None, False)
clip_region = ee.Geometry.Rectangle([-180, -30, 180, 30], None, False)

In [2]:
#树木覆盖度
vc = ee.Image('projects/ee-lmike18770616551/assets/vc_mean_ee')
# vc.projection().nominalScale()

In [3]:
#干燥指数
pet = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").filterDate(ee.Date.fromYMD(2001,1,1),ee.Date.fromYMD(2024,1,1)).select('pet').sum().multiply(0.1).multiply(1/23)
pr = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").filterDate(ee.Date.fromYMD(2001,1,1),ee.Date.fromYMD(2024,1,1)).select('pr').sum().multiply(1/23)
dry = pet.multiply(pr.pow(-1))
# geemap.download_ee_image(dry,r'D:\work\gee_\gee_download\yuan\test\dry.tif',scale= 4638.3,crs= 'EPSG:4326',region=sh)

In [4]:
#气温
temp = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")\
        .filterDate(ee.Date.fromYMD(2001,1,1),ee.Date.fromYMD(2024,1,1))\
        .mean().select(['tmmn','tmmx']).multiply(0.1)
temp = temp.expression('(tmmx+tmmn)/2',
                       {'tmmx':temp.select('tmmx'),
                        'tmmn':temp.select('tmmn')})

In [ ]:
#LAI
from datetime import datetime,timedelta
days = timedelta(days=8)
t = datetime(2001,1,1)
i = 1
LAI_8 = {}
while(t.year<=2023):
    # print(t)
    next_t = t+days
    image = ee.ImageCollection("MODIS/061/MOD15A2H").filter(ee.Filter.date(f'{t.year}-{str(t.month).rjust(2,"0")}-{str(t.day).rjust(2,"0")}', f'{next_t.year}-{str(next_t.month).rjust(2,"0")}-{str(next_t.day).rjust(2,"0")}')).select('Lai_500m')
    if image.size().getInfo()==0:
        t=next_t
        i+=1
        continue
    # geemap.download_ee_image(image.first().clip(roi),head_path+os.sep+f'LAI-{t.year}-{i}.tif',crs='EPSG:4326')
    LAI_8[f'{t.year}-{i}']=image.mean()
    if next_t.year!=t.year:
        t=datetime(next_t.year,1,1)
        i=1
        continue
    i+=1
    t=next_t
LAI_keys = list(LAI_8.keys())
LAI_p = []
LAI_ = {}
# LAI_mean = ee.Image(0)
dd = 1
p = 1
for i in range(len(LAI_keys)):
    if (i+1) == len(LAI_keys):
        pass
    else:
        per = (LAI_8[LAI_keys[i+1]].add(LAI_8[LAI_keys[i]].multiply(-1))).divide(7)
    for d in range(8):
        LAI_p = LAI_p.append(LAI_8[LAI_keys[i]].add(per.multiply(d)))
        if dd%5 == 0:
            # LAI_mean = LAI_mean.add(LAI_p.divide(5))
            LAI_[f"{LAI_keys[i].split('-')[0]}-{p}"]=ee.ImageCollection(LAI_p).mean()
            LAI_p = []
            if dd==365:
                dd = 1
                p = 1
                break
            else:
                p+=1
        dd+=1
    print(LAI_keys[i])
# LAI_mean = ee.ImageCollection(LAI_)
# LAI_mean = LAI_mean.mean()
# LAI_std = ee.Image(0)
# for i in LAI_:
#     LAI_std = LAI_std.add(LAI_[i].add(LAI_mean.multiply(-1)).pow(2))
# LAI_std.divide(len(LAI_))
# LAI = {}
# for i in LAI_:
#     LAI[i] = LAI_[i].add(LAI_mean.multiply(-1)).divide(LAI_std)

# geemap.download_ee_image(LAI[i],r'D:\work\gee_\gee_download\yuan\test\LAI.tif',scale= 4638.3,crs= 'EPSG:4326',region=sh)


In [ ]:
LAI_coll = ee.ImageCollection(list(LAI_.values()))
LAI_mean = LAI_coll.mean()
LAI_std = LAI_coll.map(lambda im: im.subtract(LAI_mean).pow(2)).sum()

# LAI_std = ee.Image(0)
# for i in LAI_:
#     LAI_std = LAI_std.add(LAI_[i].add(LAI_mean.multiply(-1)).pow(2))
# LAI_std.divide(len(LAI_))
# LAI = {}
# for i in LAI_:
#     LAI[i] = LAI_[i].subtract(LAI_mean).divide(LAI_std)
LAI = LAI_coll.map(lambda im: im.subtract(LAI_mean).divide(LAI_std))
LAI.first()
# geemap.download_ee_image(LAI.first().toFloat(),r'D:\work\gee_\gee_download\yuan\test\LAI.tif',scale= 4638.3,crs= 'EPSG:4326',region=sh_cn)


In [ ]:
#多轴图
p_dry = ee.Image(40)
p_dry = p_dry.add(dry.lt(0.5).multiply(-10))
p_dry = p_dry.add(dry.lt(1).multiply(-10))
p_dry = p_dry.add(dry.lt(2).multiply(-10))
p_dry = p_dry.multiply(dry.lt(4))

p_tc = ee.Image(4)
p_tc = p_tc.add(vc.lt(0.25).multiply(-1))
p_tc = p_tc.add(vc.lt(0.5).multiply(-1))
p_tc = p_tc.add(vc.lt(0.75).multiply(-1))
p_tc = p_tc.multiply(vc.lt(1))

p_tc = p_tc.multiply(p_dry.gt(0.5))
p_dry = p_dry.multiply(p_tc.gt(0.5))
p_arr = p_dry.add(p_tc)
# geemap.download_ee_image(p_arr,r'D:\work\gee_\gee_download\yuan\test\p_arr.tif',scale= 4638.3,crs= 'EPSG:4326',region=sh)

In [ ]:
import datetime

quans = {}
soil30 = []
star_time = 2004
end_time = 2024
clip30 = ee.Image(1)
# clip30.clip(clip_region)
p = 1
for day in range(0, 365, 5):
    swvl_day_list = []
    for year in range(star_time, end_time):
        s = datetime.datetime(year, 1, 1)
        image =  ee.ImageCollection("NASA/GLDAS/V022/CLSM/G025/DA1D").filterDate((s+datetime.timedelta(day)).strftime("%Y-%m-%d"), 
                                                                        (s+datetime.timedelta(day+5)).strftime("%Y-%m-%d")).mean()
        swvl = image.select('SoilMoist_RZ_tavg')
        # swvl = image.expression('swvl1*c1+swvl2*c2+swvl3*c3',
        #                          {'swvl1':image.select('soil_temperature_level_1'),
        #                           'swvl2':image.select('soil_temperature_level_2'),
        #                           'swvl3':image.select('soil_temperature_level_3'),
        #                           'c1':0.2333,'c2':0.7,'c3':0.0667})
        swvl_day_list.append(swvl)
    
    swvl_day_images = ee.ImageCollection(swvl_day_list)
    
    for year in range(star_time, end_time):
        ob_im = swvl_day_list[year - star_time]
        quan = swvl_day_images.map(lambda img: ob_im.gt(img)).sum().add(1).divide(end_time-star_time)
        if (day < 60 or day >304):
            quan = quan.multiply(0)
            # quan = quan.multiply(clip30.clip(clip_region))
        quans[f"{year}-{p}"] = quan
 
    p+=1

# geemap.download_ee_image(quans['2001-30'],r'D:\work\gee_\gee_download\yuan\test\quan.tif',scale= 27830,crs= 'EPSG:4326',region=sh,max_tile_dim=100)

In [ ]:
mask = Image(1)
mask = mask.where(dry.gt(4),0).where(temp.lt(0),0)

In [ ]:
dry_index_year = []

t_all = ee.Image(0)
count_all = ee.Image(0)
for year in range(2004,2024):
    dry_index = ee.Image(0)
    count = ee.Image(0)
    switch = ee.Image(0)
    st_switch = ee.Image(0)
    st_count = ee.Image(0)
    ed_switch = ee.Image(0)
    ed_count = ee.Image(0)
    LAI_switch = ee.Image(0)
    per_count = ee.Image(0)
    for p in range(1,74):
        arr = quans[f'{year}-{p}'].where(mask.eq(0),0)
        LAI_arr = LAI[f'{year}-{p}'].where(mask.eq(0),0)
        st_switch = st_switch.where(st_count.gt(5),0)
        st_count = st_count.multiply(st_switch)
        ed_count = ed_count.where((switch.multiply(ed_switch)).gt(0),-3)
        ed_switch = ed_switch.where(arr.gte(0.2).multiply(switch),1)
        ed_switch = ed_switch.where(arr.lt(0.2).multiply(switch),0)
        ed_count = ed_count.where(arr.lt(0.2).multiply(switch),0)
        st_switch = st_switch.where(arr.gte(0.4),1)
        st_count = st_count.where(arr.gte(0.4),0)
        st_count = st_count.add(st_switch)
        ed_count = ed_count.add(ed_switch)
        switch = switch.where(ed_count.eq(-2).multiply(ed_switch),0)
        count = count.add(ed_count.eq(-2).multiply(ed_switch).multiply(-1))
        count_all = count_all.add(count.gte(6).multiply(count.lte(18)).multiply(ed_count.eq(-2)).multiply(ed_switch).multiply(per_count.neq(count)).multiply(per_count.neq(0)))
        t_all = t_all.add(per_count.multiply(count.gte(6).multiply(count.lte(18)).multiply(ed_count.eq(-2)).multiply(ed_switch).multiply(per_count.neq(count)).multiply(per_count.neq(0))))
        # dry_index = dry_index.add(count.gte(6).multiply(count.lte(18)).multiply(ed_count.eq(-2)).multiply(ed_switch))
        LAI_switch = LAI_switch.where(ed_count.eq(-2).multiply(ed_switch),0)
        per_count = per_count.where(ed_count.eq(-2).multiply(ed_switch),0)
        count = count.where(ed_count.eq(-2).multiply(ed_switch),0)
        ed_switch = ed_switch.where(ed_count.eq(-2).multiply(ed_switch),0)
        ed_count = ed_count.where(ed_count.eq(-2).multiply(ed_switch),0)
        switch = switch.where(arr.lt(0.2).multiply(st_switch),1)
        LAI_switch = LAI_switch.where(arr.lt(0.2).multiply(st_switch),1)
        LAI_switch = LAI_switch.where(LAI_arr.lt(0),0)
        per_count = per_count.add(LAI_switch)
        st_switch = st_switch.where(arr.lt(0.2).multiply(st_switch),0)
        count = count.add(switch)
    # geemap.download_ee_image(dry_index,f'D:\\work\\gee_\\gee_download\\yuan\\test\\dry_index-{year}.tif',scale=27830, crs='epsg:4326',region=sh, max_tile_dim=100)
    # geemap.ee_export_image_to_asset(dry_index,f'dry_index-{year}-{p}',f'projects/ee-lmike18770616551/assets/image/dry_index-{year}-{p}',scale=27830,region=sh,crs='epsg:4326')
    # print(f'dry_index-{year}-{p}')
    dry_index_year.append(dry_index)
    # break


In [ ]:

# dry_index = dry_index_year[0]
# for dry_year in dry_index_year[1:5]:
#     dry_index = dry_index.add(dry_year)
# geemap.download_ee_image(dry_index,f'D:\\work\\gee_\\gee_download\\yuan\\test\\dry_index.tif',scale=27830, crs='epsg:4326',region=sh, max_tile_dim=100)
# dry_index = dry_index_year[5]
# for dry_year in dry_index_year[6:10]:
#     dry_index = dry_index.add(dry_year)
# geemap.download_ee_image(dry_index,f'D:\\work\\gee_\\gee_download\\yuan\\test\\dry_index1.tif',scale=27830, crs='epsg:4326',region=sh, max_tile_dim=100)
# dry_index = dry_index_year[10]
# for dry_year in dry_index_year[11:15]:
#     dry_index = dry_index.add(dry_year)
# geemap.download_ee_image(dry_index,f'D:\\work\\gee_\\gee_download\\yuan\\test\\dry_index2.tif',scale=27830, crs='epsg:4326',region=sh, max_tile_dim=100)
dry_index = dry_index_year[15]
for dry_year in dry_index_year[16:20]:
    dry_index = dry_index.add(dry_year)
geemap.download_ee_image(dry_index,f'D:\\work\\gee_\\gee_download\\yuan\\test\\dry_index3.tif',scale=27830, crs='epsg:4326',region=sh, max_tile_dim=100)
dry_index = dry_index_year[20]
for dry_year in dry_index_year[21:]:
    dry_index = dry_index.add(dry_year)
geemap.download_ee_image(dry_index,f'D:\\work\\gee_\\gee_download\\yuan\\test\\dry_index4.tif',scale=27830, crs='epsg:4326',region=sh, max_tile_dim=100)


In [ ]:
# Map.addLayer(dry_index, {'max': 10, 'min': 0}, "dry_index")